# Hands-on Paper Hypothesis Assistant

In [1]:
!pip install torch --index-url https://download.pytorch.org/whl/cpu
!pip install sentence-transformers
!pip install huggingface_hub

Looking in indexes: https://download.pytorch.org/whl/cpu
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.4/190.4 MB 2.1 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.2.1+cu121
    Uninstalling torch-2.2.1+cu121:
      Successfully uninstalled torch-2.2.1+cu121
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.3.0+cpu which is incompatible.
torchtext 0.17.1 requires torch==2.2.1, but you have torch 2.3.0+cpu which is incompatible.
torchvision 0.17.1+cu121 requires torch==2.2.1, but you have torch 2.3.0+cpu which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.5 MB/s eta 0:00:00


In [2]:
#let's download data and load the model from Huggingface

import os
cwd = os.getcwd()
from huggingface_hub import hf_hub_download
hf_hub_download(repo_id="eaglew/COLING2024", local_dir=cwd, filename="train.json")
hf_hub_download(repo_id="eaglew/COLING2024", local_dir=cwd, filename="test.json")
hf_hub_download(repo_id="eaglew/COLING2024", local_dir=cwd, filename="BEST.pth")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


train.json:   0%|          | 0.00/2.32M [00:00<?, ?B/s]

test.json:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

BEST.pth:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

'/content/BEST.pth'

This tutorial will have 2 steps:
1. Retrieve inspirations (related entity) based on cosine similarity
2. Generate new hypothesis based on the inspirations  


In [3]:
import random
import numpy as np
from typing import Any, Callable, Dict, Iterable, List, Optional, Tuple, Union
from transformers import T5Tokenizer


import torch
import torch.utils.checkpoint
from torch import nn
from torch.nn import CrossEntropyLoss, BCEWithLogitsLoss
import torch.nn.functional as F

from transformers.modeling_outputs import ModelOutput, BaseModelOutputWithPastAndCrossAttentions, BaseModelOutput, Seq2SeqModelOutput, Seq2SeqLMOutput
from transformers.utils import logging

from transformers.models.t5.modeling_t5 import (
    T5Attention,
    T5LayerSelfAttention,
    T5LayerCrossAttention,
    T5Block,
    T5Stack,
    T5ForConditionalGeneration,
    T5Config,
    T5LayerNorm,
)
import copy
import warnings
import math
import pickle
from torch.utils.checkpoint import checkpoint
from dataclasses import dataclass


import torch.nn.functional as F
logger = logging.get_logger(__name__)

@dataclass
class ContrastiveSeq2SeqModelOutput(ModelOutput):
    last_hidden_state: torch.FloatTensor = None
    past_key_values: Optional[Tuple[Tuple[torch.FloatTensor]]] = None
    decoder_hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    decoder_attentions: Optional[Tuple[torch.FloatTensor]] = None
    cross_attentions: Optional[Tuple[torch.FloatTensor]] = None
    encoder_last_hidden_state: Optional[torch.FloatTensor] = None
    encoder_hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    encoder_attentions: Optional[Tuple[torch.FloatTensor]] = None
    retrieve_last_hidden_states: Optional[torch.FloatTensor] = None

@dataclass
class ContrastiveSeq2SeqLMOutput(ModelOutput):

    loss: Optional[torch.FloatTensor] = None
    mlm_loss: Optional[torch.FloatTensor] = None
    cl_loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    past_key_values: Optional[Tuple[Tuple[torch.FloatTensor]]] = None
    decoder_hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    decoder_attentions: Optional[Tuple[torch.FloatTensor]] = None
    cross_attentions: Optional[Tuple[torch.FloatTensor]] = None
    encoder_last_hidden_state: Optional[torch.FloatTensor] = None
    encoder_hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    encoder_attentions: Optional[Tuple[torch.FloatTensor]] = None



class ContrastiveHead(nn.Module):
    """Head for sentence-level classification tasks."""

    def __init__(
        self,
        inner_dim: int,
        pooler_dropout: float,
    ):
        super().__init__()
        self.dropout = nn.Dropout(p=pooler_dropout)
        self.out_proj = nn.Linear(inner_dim, 1)

    def forward(self, hidden_states: torch.Tensor, masks: torch.Tensor) -> torch.Tensor:
        hidden_states = self.out_proj(hidden_states)
        hidden_states = self.avg_pool(hidden_states, masks)
        hidden_states = torch.sigmoid(hidden_states)
        return hidden_states

    def avg_pool(self, hidden_states, mask):
        length = torch.sum(mask, 1, keepdim=True).float()
        mask = mask.unsqueeze(2)
        hidden = hidden_states.masked_fill(mask == 0, 0.0)
        avg_hidden = torch.sum(hidden, 1) / length

        return avg_hidden

__HEAD_MASK_WARNING_MSG = """
The input argument `head_mask` was split into two arguments `head_mask` and `decoder_head_mask`. Currently,
`decoder_head_mask` is set to copy `head_mask`, but this feature is deprecated and will be removed in future versions.
If you do not want to use any `decoder_head_mask` now, please set `decoder_head_mask = torch.ones(num_layers,
num_heads)`.
"""

class ContrastiveT5ForConditionalGeneration(T5ForConditionalGeneration):

    def __init__(self, config: T5Config):
        super().__init__(config)
        self.model_dim = config.d_model

        self.shared = nn.Embedding(config.vocab_size, config.d_model)

        encoder_config = copy.deepcopy(config)
        encoder_config.is_decoder = False
        encoder_config.use_cache = False
        encoder_config.is_encoder_decoder = False
        self.encoder = T5Stack(encoder_config, self.shared)

        decoder_config = copy.deepcopy(config)
        decoder_config.is_decoder = True
        decoder_config.is_encoder_decoder = False
        decoder_config.num_layers = config.num_decoder_layers
        self.decoder = T5Stack(decoder_config, self.shared)

        self.lm_head = nn.Linear(config.d_model, config.vocab_size, bias=False)

        self.contrastive_head = ContrastiveHead(config.d_model, config.dropout_rate)

        # Initialize weights and apply final processing
        self.post_init()

        # Model parallel
        self.model_parallel = False
        self.device_map = None

    def forward(
        self,
        input_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.FloatTensor] = None,

        neg_ids: Optional[torch.LongTensor] = None,
        neg_num_total: Optional[int]=1,
        valid: Optional[bool] = False,
        ppl: Optional[bool] = False,

        decoder_input_ids: Optional[torch.LongTensor] = None,
        decoder_attention_mask: Optional[torch.BoolTensor] = None,
        head_mask: Optional[torch.FloatTensor] = None,
        decoder_head_mask: Optional[torch.FloatTensor] = None,
        cross_attn_head_mask: Optional[torch.Tensor] = None,
        encoder_outputs: Optional[Tuple[Tuple[torch.Tensor]]] = None,
        past_key_values: Optional[Tuple[Tuple[torch.Tensor]]] = None,
        inputs_embeds: Optional[torch.FloatTensor] = None,
        decoder_inputs_embeds: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
        use_cache: Optional[bool] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple[torch.FloatTensor], Seq2SeqLMOutput]:
        use_cache = use_cache if use_cache is not None else self.config.use_cache
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        # FutureWarning: head_mask was separated into two input args - head_mask, decoder_head_mask
        if head_mask is not None and decoder_head_mask is None:
            if self.config.num_layers == self.config.num_decoder_layers:
                warnings.warn(__HEAD_MASK_WARNING_MSG, FutureWarning)
                decoder_head_mask = head_mask

        # Encode if needed (training, first prediction pass)
        if encoder_outputs is None:
            # Convert encoder inputs in embeddings if needed
            encoder_outputs = self.encoder(
                input_ids=input_ids,
                attention_mask=attention_mask,
                inputs_embeds=inputs_embeds,
                head_mask=head_mask,
                output_attentions=output_attentions,
                output_hidden_states=output_hidden_states,
                return_dict=return_dict,
            )
        elif return_dict and not isinstance(encoder_outputs, BaseModelOutput):
            encoder_outputs = BaseModelOutput(
                last_hidden_state=encoder_outputs[0],
                hidden_states=encoder_outputs[1] if len(encoder_outputs) > 1 else None,
                attentions=encoder_outputs[2] if len(encoder_outputs) > 2 else None,
            )

        hidden_states = encoder_outputs[0]

        if self.model_parallel:
            torch.cuda.set_device(self.decoder.first_device)

        if labels is not None and decoder_input_ids is None and decoder_inputs_embeds is None:
            # get decoder inputs from shifting lm labels to the right
            decoder_input_ids = self._shift_right(labels)

        # Set device for model parallelism
        if self.model_parallel:
            torch.cuda.set_device(self.decoder.first_device)
            hidden_states = hidden_states.to(self.decoder.first_device)
            if decoder_input_ids is not None:
                decoder_input_ids = decoder_input_ids.to(self.decoder.first_device)
            if attention_mask is not None:
                attention_mask = attention_mask.to(self.decoder.first_device)
            if decoder_attention_mask is not None:
                decoder_attention_mask = decoder_attention_mask.to(self.decoder.first_device)

        # Decode
        decoder_outputs = self.decoder(
            input_ids=decoder_input_ids,
            attention_mask=decoder_attention_mask,
            inputs_embeds=decoder_inputs_embeds,
            past_key_values=past_key_values,
            encoder_hidden_states=hidden_states,
            encoder_attention_mask=attention_mask,
            head_mask=decoder_head_mask,
            cross_attn_head_mask=cross_attn_head_mask,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        sequence_output = decoder_outputs[0]

        # Set device for model parallelism
        if self.model_parallel:
            torch.cuda.set_device(self.encoder.first_device)
            self.lm_head = self.lm_head.to(self.encoder.first_device)
            sequence_output = sequence_output.to(self.lm_head.weight.device)

        if self.config.tie_word_embeddings:
            # Rescale output before projecting on vocab
            # See https://github.com/tensorflow/mesh/blob/fa19d69eafc9a482aff0b59ddd96b025c0cb207d/mesh_tensorflow/transformer/transformer.py#L586
            sequence_output = sequence_output * (self.model_dim**-0.5)

        lm_logits = self.lm_head(sequence_output)

        masked_lm_loss = None
        contrastive_loss = None
        loss = None
        if labels is not None:
            if ppl:
                loss_fct = CrossEntropyLoss(ignore_index=-100, reduction='sum')
                ppl_list = []
                for k in range(labels.size(0)):
                    masked_lm_loss = loss_fct(lm_logits[k,:,:], labels[k,:])
                    # prob = torch.exp(-masked_lm_loss)
                    prob = -masked_lm_loss
                    ppl_list.append(prob.item())
                return ppl_list
            loss_fct = CrossEntropyLoss(ignore_index=-100)
            masked_lm_loss = loss_fct(lm_logits.view(-1, lm_logits.size(-1)), labels.view(-1))

            if not valid:
                contrastive_loss = self.contrastive(
                    decoder_outputs,
                    labels,
                    attention_mask,
                    neg_ids,
                    neg_num_total,
                    encoder_outputs,
                    decoder_attention_mask,
                    past_key_values,
                    decoder_inputs_embeds,
                    decoder_head_mask,
                    output_attentions,
                    cross_attn_head_mask,
                    output_hidden_states,
                    return_dict,
                    use_cache)
                loss = masked_lm_loss + contrastive_loss
            else:
                loss = masked_lm_loss

        if not return_dict:
            output = (lm_logits,) + decoder_outputs[1:] + encoder_outputs
            return ((loss,masked_lm_loss,contrastive_loss,) + output) if loss is not None else output

        return ContrastiveSeq2SeqLMOutput(
            loss=loss,
            mlm_loss=masked_lm_loss,
            cl_loss=contrastive_loss,
            logits=lm_logits,
            past_key_values=decoder_outputs.past_key_values,
            decoder_hidden_states=decoder_outputs.hidden_states,
            decoder_attentions=decoder_outputs.attentions,
            cross_attentions=decoder_outputs.cross_attentions,
            encoder_last_hidden_state=encoder_outputs.last_hidden_state,
            encoder_hidden_states=encoder_outputs.hidden_states,
            encoder_attentions=encoder_outputs.attentions,
        )

    def contrastive(
        self,
        decoder_outputs,
        labels,
        attention_mask,
        neg_ids,
        expand_size,
        encoder_outputs,
        decoder_attention_mask,
        past_key_values,
        decoder_inputs_embeds,
        decoder_head_mask,
        output_attentions,
        cross_attn_head_mask,
        output_hidden_states,
        return_dict,
        use_cache
    ):
        pos_label_mask = labels != -100
        pos_emb = self.contrastive_head(decoder_outputs.last_hidden_state, pos_label_mask)


        decoder_input_ids = self._shift_right(neg_ids)
        bs = labels.size(0)

        expanded_return_idx = (
            torch.arange(attention_mask.shape[0]).view(-1, 1).repeat(1, expand_size).view(-1).to(attention_mask.device)
        )

        encoder_outputs["last_hidden_state"] = encoder_outputs.last_hidden_state.index_select(
            0, expanded_return_idx.to(encoder_outputs.last_hidden_state.device)
        )

        attention_mask = attention_mask.index_select(0, expanded_return_idx).to(attention_mask.device)


        decoder_outputs = self.decoder(
            input_ids=decoder_input_ids,
            attention_mask=decoder_attention_mask,
            inputs_embeds=decoder_inputs_embeds,
            past_key_values=past_key_values,
            encoder_hidden_states=encoder_outputs.last_hidden_state,
            encoder_attention_mask=attention_mask,
            head_mask=decoder_head_mask,
            cross_attn_head_mask=cross_attn_head_mask,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )


        neg_label_mask = neg_ids != self.config.pad_token_id
        neg_emb = self.contrastive_head(decoder_outputs.last_hidden_state, neg_label_mask).view(bs, expand_size)


        loss_fct = CrossEntropyLoss()
        all_logit = torch.cat([pos_emb,neg_emb], dim=1)
        l = torch.zeros([bs], dtype=torch.long, device=neg_emb.device)
        cl_loss = loss_fct(all_logit, l)
        return cl_loss

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

First, we will try to encode all instances in the training set with sentence transformer

In [4]:
from sentence_transformers import SentenceTransformer, util
import json
from collections import defaultdict
from tqdm import tqdm
import copy
import os
import shutil


In [5]:
#In this example, we're going to use 'all-MiniLM-L6-v2' to encode all sentences


model_sent = SentenceTransformer('all-MiniLM-L6-v2')

context2cands = defaultdict(dict)
context2c = defaultdict(set)
context2ids = {}
contexts = []

in_name = 'train.json'
with open(in_name, 'r') as file_j:
    for idxn, line in tqdm(enumerate(file_j), "Encoding"):
        pdf_dict = json.loads(line)
        input_, context_ = pdf_dict['input'].split('| context: ')
        type_ = input_.split()[-1]
        context = input_ + ' context: ' + context_
        context2ids[context] = pdf_dict['id']
        context2cands[context][pdf_dict['output']] = type_
        context2c[context].add(context_)
        contexts.append(context)
print('finish loading')
context_embeddings = model_sent.encode(contexts, batch_size=10, convert_to_tensor=True,show_progress_bar=True)

# this is the function that help us find ideas proposed for related problems in the training set
def get_retrieve(current_context, contexts, context_embeddings, type_):
    c_embedding = model_sent.encode([current_context], convert_to_tensor=True)
    hits = util.semantic_search(c_embedding, context_embeddings, top_k=100)
    retrieve = {}
    count = 0
    for i in range(20):
        cur_id = hits[0][i]['corpus_id']
        context = contexts[cur_id]
        retrieve.update(context2cands[context])

    entities = []
    for entity in retrieve:
        if retrieve[entity] == type_:
            entities.append(entity)


    return entities

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Encoding: 2000it [00:00, 6550.07it/s]

finish loading


Batches:   0%|          | 0/200 [00:00<?, ?it/s]

In [7]:
#let's load the model from Huggingface

import torch
from transformers import AutoTokenizer

tokenizer =  AutoTokenizer.from_pretrained("t5-large")
PATH = "BEST.pth"
device = torch.device('cpu')
model = ContrastiveT5ForConditionalGeneration.from_pretrained("t5-large")

model.load_state_dict(torch.load(PATH, map_location=device))
filename = "test.json"
test_dataset = []
with open(filename, 'r') as f:
    for line in tqdm(f):
        test_dataset.append(json.loads(line))

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

Some weights of ContrastiveT5ForConditionalGeneration were not initialized from the model checkpoint at t5-large and are newly initialized: ['contrastive_head.out_proj.bias', 'contrastive_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

5309it [00:00, 54023.87it/s]


In [8]:
#let's randomly select a test case from the test set and try to retreive related inspirations from background context and seed term

import random
cur_data = random.choice(test_dataset)

input_ = cur_data["input"]
target = cur_data["rel_sent"].lower()
ids = cur_data["id"]
year = cur_data["year"]
input_, context_ = cur_data['input'].split('| context: ')
type_ = input_.split()[-1]
tmp = copy.deepcopy(pdf_dict)
current_context = input_ + ' context: ' + context_
retrieved_entity = get_retrieve(current_context, contexts, context_embeddings, type_)
new_input  = input_ + ' | retrieve: ' + ', '.join(retrieved_entity) + ' | context: ' + context_
print('seed term:', input_)
print('background context:', context_)
print('semantic related entities:', ', '.join(retrieved_entity))

seed term: automatic speech recognition is used for Task
background context: transcription is often reported as the bottleneck in endangered language documentation , requiring large efforts from scarce speakers and transcribers . in general , automatic speech recognition ( asr ) can be accurate enough to accelerate transcription only if trained on large amounts of transcribed data . however , when a single speaker is involved , several studies have reported encouraging results for phonetic transcription even with small amounts of training .
semantic related entities: english mispronunciation detection, automatic speech recognition, endangered language documentation, non - english auxiliary tasks, direct speech translation


In [12]:
# let's generate hypothesis based on the input
source_id = tokenizer(new_input, truncation=True, max_length=512).input_ids
input_ids = torch.LongTensor([source_id])
output = model.generate(input_ids = input_ids,
                max_length=100,
                repetition_penalty=1.5,
                num_beams=10)
predictions = tokenizer.batch_decode(output, skip_special_tokens=True)
print(predictions[0])

automatic speech recognition for endangered language transcription.
